In [1]:
import pandas as pd 
from sdv.lite import SingleTablePreset
from copulas.multivariate import VineCopula
import numpy as np

In [3]:
df = pd.read_csv('../kaggle/data/train-metadata.csv')

/tmp/ipykernel_80654/4075934004.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../kaggle/data/train-metadata.csv')


In [4]:
mal_df = df[df['target'] == 1]
cols_to_drop = ["isic_id", "patient_id", "mel_thick_mm", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "attribution", "copyright_license", "tbp_lv_location", "tbp_lv_location_simple", "tbp_lv_dnn_lesion_confidence", "anatom_site_general", "image_type", "tbp_tile_type"]
mal_df.drop(columns=cols_to_drop, inplace=True)
mal_df.dropna(inplace=True)
mal_df.drop(columns=["target"], inplace=True)
mal_df["sex"] = mal_df["sex"].replace({"male": 1, "female": 0})

# Drop any column with a string value or NaN
for col in mal_df.columns:
    if mal_df[col].dtype == "object":
        mal_df.drop(columns=[col], inplace=True)

In [5]:
dist = VineCopula("direct")
dist.fit(mal_df)

/home/ubuntu/miniconda3/envs/tf-v2/lib/python3.8/site-packages/copulas/multivariate/vine.py:73: UserWarning: Vines have not been fully tested on Python >= 3.8 and might produce wrong results.
  warnings.warn(
/home/ubuntu/miniconda3/envs/tf-v2/lib/python3.8/site-packages/copulas/bivariate/base.py:162: RuntimeWarning: Data does not appear to be uniform.
  warnings.warn('Data does not appear to be uniform.', category=RuntimeWarning)


In [6]:
samples = dist.sample(10000)

In [7]:
samples = pd.DataFrame(samples, columns=mal_df.columns)
samples["target"] = 1


In [8]:
samples.head()
samples.to_csv("mal_samples.csv", index=False)

,age_approx,sex,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,...,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,target
0,51.854976,-0.126443,4.348293,18.920550,10.366828,19.471754,22.912010,32.065529,31.242137,43.781183,...,23.258421,0.226008,4.835596,1.128741,0.107662,-4.807688,-237.142448,384.414622,-97.655779,1
1,32.529837,0.089858,8.557781,11.015516,10.575277,20.854608,22.487333,31.623733,25.552457,51.102559,...,5.588584,2.064648,4.996369,2.094061,0.015293,136.642487,220.229366,1267.021809,157.624922,1
2,41.751140,-0.015992,11.600520,16.132921,14.829224,20.964391,18.952173,34.278714,27.988465,29.036563,...,22.136954,2.864062,10.324304,1.799545,0.105315,20.439375,-467.652082,1149.436877,-2.347786,1
3,63.085577,-0.011630,15.686968,17.981337,16.772677,23.028741,24.138059,36.697134,27.447898,53.757526,...,24.970557,5.867677,3.732133,1.630191,0.171754,46.327870,9.017330,595.296743,31.423314,1
4,72.014035,0.232172,9.496940,21.357734,16.170969,27.072000,17.780690,39.614932,27.494865,49.304447,...,24.476897,3.003767,0.927128,1.724050,0.223287,3.542201,-180.511257,1072.685902,-58.035930,1
